In [1]:
import cv2
import pytesseract
from PIL import Image
import re
from datetime import datetime, timedelta
import numpy as np
import pandas as pd
import os

In [2]:
filex = r'old_puzzles/180.png'


In [3]:
mydir = 'old_puzzles/'
all_row_list = []

for file in os.listdir(mydir):
    row_list=[]
    img = cv2.imread(mydir + file)
    puzzle_number_crop = img[180:220, 850:1082]
    text= pytesseract.image_to_string(puzzle_number_crop)
    regex = re.compile('[^0-9]')
    puzzle_number = regex.sub('', text)
    puzzle_number = int(puzzle_number)

    first_plusword_date = '2022-05-23'
    first_plusword_date = datetime.fromisoformat(first_plusword_date)
    plusword_date = first_plusword_date +  timedelta(days=puzzle_number - 1)
    plusword_date=plusword_date.strftime("%Y-%m-%d")
    row_list.append(plusword_date)
    row_list.append(puzzle_number)

    plusword_crop = img[746:810, 690:1150]
    text= pytesseract.image_to_string(plusword_crop)
    regex = re.compile('[^A-Z]')
    plusword_solution = regex.sub('', text)
    row_list.append(plusword_solution)

    grid_crop = img[230:705, 690:1150]

    answer_list= []

    row_coordinates = [grid_crop[40:90, 10:500],
                       grid_crop[120:190, 10:500],
                       grid_crop[210:290, 10:500],
                       grid_crop[300:380, 10:500],
                       grid_crop[390:480, 10:500]]

    for row_coordinate in row_coordinates:
        row_crop = row_coordinate
        text= pytesseract.image_to_string(row_crop)

        regex = re.compile('[^A-Z]')
        text = regex.sub('', text)
        answer_list.append(text)

    row_list.append(answer_list)

    question_crop = img[250:680, 1150:1800]
    text= pytesseract.image_to_string(question_crop)
    text = text.splitlines()
    regex = re.compile('[^a-z A-Z]')
    text = [regex.sub('', x) for x in text]
    questions = [x for x in text if x]

    row_list.append(questions)

    colour_check_coordinates=[(100 ,87),
                              (100, 179),
                              (100 ,270),
                              (100 ,365),
                              (100 ,456),
                              (195 ,87),
                              (195 ,179),
                              (195 ,270),
                              (195 ,365),
                              (195 ,456),
                              (285 ,87),
                             (285 ,179),
                             (285 ,270),
                             (285 ,365),
                             (285 ,456),
                             (380 ,87),
                             (380 ,179),
                             (380 ,270),
                             (380 ,365),
                             (380 ,456),
                             (459 ,87),
                             (459 ,179),
                             (459 ,270),
                             (459 ,365),
                             (459 ,456)]

    white_rgb = np.array([255, 255, 255])
    yellow_rgb =np.array([223, 253, 255])
    green_rgb = np.array([218, 241, 214])

    green = []
    yellow= []
    i = 0
    for cell in colour_check_coordinates:
        i += 1
        colour_arr = grid_crop[cell]

        if not (colour_arr==white_rgb).all():
            if (colour_arr==yellow_rgb).all():
                yellow.append(i)
            if (colour_arr==green_rgb).all():
                green.append(i)

    row_list.append(str(yellow))
    row_list.append(str(green))
    all_row_list.append(row_list)

In [4]:
df = pd.DataFrame(all_row_list)

In [5]:
df

,0,1,2,3,4,5,6
0,2022-11-18,180,OD,"[MADAM, OXIDIE, AISLE, TACT, SLOBS]","[ tron rust, Gangway, Implied, Layabouts, ...","[3, 8, 9, 12]","[6, 19, 23]"
1,2022-08-12,82,BLESS,"[SORTS, QUART, UTTER, ADIEU, BOOOST]","[Across, Separates into groups, Two pints, ...","[1, 14, 19]","[5, 21, 24]"
2,2022-09-02,103,TIMD,"[PANO, OLDEN, MEOWS, PUREE, SMELT]","[Across, Softly, Former past, Catty noises, ...","[8, 11, 22, 25]",[2]
3,2023-03-20,302,REPEUL,"[CHOPS, RAD, NEPT, COOEE, KNDS]","[Across, Cuts into small pieces, Geometrical...","[4, 13, 14, 20]","[6, 19]"
4,2022-05-23,1,LURID,"[CAMEL, OVIATIE, MOCHA, MIRES, ADORE]","[Across, The ship of the desert, Broadest bel...","[5, 17, 22, 24]",[18]
...,...,...,...,...,...,...,...
333,2022-10-09,140,TRASH,"[SCRAPI, CHAIR, RONDO, URGED, BEERS]","[Across, Discard, Seat with a back, Type of...","[1, 3, 4, 7, 10, 11, 24, 25]","[8, 17]"
334,2022-12-17,209,GLAND,"[LEASTI, UNDER, FRODO, FORGO, SLEEP]","[Across, x , , Minimum, Ina state of unconsci...","[1, 7, 8, 14, 19]","[3, 22]"
335,2022-08-30,100,BIGOT,"[GHANAI, REBEL, ALIVIE, DER, EXERT]","[Across, African country, Revolutionary, Not ...","[1, 8, 13]","[17, 25]"
336,2022-07-12,51,PRIDE,"[CADET, LILAMA, ALIAS, POSIT, STYLE]","[Across, Apprentice trainee, Woolly animal, ...","[3, 4, 19]","[13, 16, 25]"


In [6]:
df.to_csv('test.csv', index=False)